In [1]:
import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
from torch.utils.data import Dataset
import os
import pandas as pd
import requests
from tqdm.auto import tqdm
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import re

/apps/Arch/software/Python/3.8.6-GCCcore-10.2.0/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
model_name = 'Rostlab/prot_bert_bfd'
import os
os.chdir("/mimer/NOBACKUP/groups/snic2019-35-21/PPI/pickle_files/")

In [3]:
class DeepLocDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, split="train", tokenizer_name='Rostlab/prot_bert_bfd', max_length=2048):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.datasetFolderPath = ''
        self.trainFilePath = os.path.join(self.datasetFolderPath, 'combined_train.csv')
        self.valFilePath = os.path.join(self.datasetFolderPath, 'combined_val.csv')
        self.testFilePath  = os.path.join(self.datasetFolderPath, 'combined_test.csv')

        

        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, do_lower_case=False)

        if split=="train":
          self.seqs, self.labels = self.load_dataset(self.trainFilePath)
        elif split == "val":
          self.seqs, self.labels = self.load_dataset(self.valFilePath)
        else: 
            self.seqs, self.labels = self.load_dataset(self.testFilePath)
            

        self.max_length = max_length

  

    def load_dataset(self,path):
       
      
        df = pd.read_csv(path)
        df = df.sample(frac=1).reset_index(drop=True)
        
        seq = list(df['combined_pro'])
        label = list(df['label'])

        assert len(seq) == len(label)
        return seq, label

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        seq = " ".join("".join(self.seqs[idx].split()))
        seq = re.sub(r"[UZOB]", "X", seq)

        seq_ids = self.tokenizer(seq, truncation=True, padding='max_length', max_length=self.max_length)

        sample = {key: torch.tensor(val) for key, val in seq_ids.items()}
        sample['labels'] = torch.tensor(self.labels[idx])

        return sample

In [4]:
train_dataset = DeepLocDataset(split="train", tokenizer_name=model_name, max_length=2048) # max_length is only capped to speed-up example.
val_dataset = DeepLocDataset(split="valid", tokenizer_name=model_name, max_length=2048)
test_dataset = DeepLocDataset(split="test", tokenizer_name=model_name, max_length=2048)

In [5]:
def model_init():
  return AutoModelForSequenceClassification.from_pretrained(model_name)

In [6]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
     

In [7]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=30,              # total number of training epochs
    per_device_train_batch_size=1,   # batch size per device during training
    per_device_eval_batch_size=10,   # batch size for evaluation
    warmup_steps=1000,    # number of warmup steps for learning rate scheduler
    learning_rate = 0.00007,
    weight_decay=0.0001,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=200,               # How often to print logs
    do_train=True,                   # Perform training
    do_eval=True,                    # Perform evaluation
    evaluation_strategy="epoch",     # evalute after eachh epoch
    gradient_accumulation_steps=64,  # total number of steps before back propagation
    fp16=True,                       # Use mixed precision
    fp16_opt_level="02",             # mixed precision mode
    run_name="ProBert-BFD-MS",       # experiment name
    seed=3                           # Seed for experiment reproducibility 3x3
)

trainer = Trainer(
    model_init=model_init,                # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics = compute_metrics,    # evaluation metrics
)

trainer.train()

loading configuration file https://huggingface.co/Rostlab/prot_bert_bfd/resolve/main/config.json from cache at /cephyr/users/anwer/Alvis/.cache/huggingface/transformers/67f460acc7e7e147ff828e909ffe419d00d66ce679c682bc4ab715c107bcbe41.baf557855a8618d0ddfb6c23bfd135bfc38ccf8c3fb099b8df45eb110ccf05e9
Model config BertConfig {
  "_name_or_path": "Rostlab/prot_bert_bfd",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 40000,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 30,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30
}

loading weights file https://huggingface.co/Rostlab/prot_bert_bfd/resol

AttributeError: 'NoneType' object has no attribute 'message_types_by_name'

In [ ]:
!pwd

In [16]:
!cd /mimer/NOBACKUP/groups/snic2019-35-21/PPI/pickle_files/

In [19]:
!pwd

/mimer/NOBACKUP/groups/snic2019-35-21/PPI/pickle_files


In [20]:
!ls

 amino_acids_benchmark_embedded_vectors_bert_albert.pickle
 amino_acids_benchmark_embedded_vectors_bert_bfd.pickle
 amino_acids_embedded_vectors_bert_bfd.pickle
 benchmark_dataset_prot_seq.pickle
 combined_test.csv
 combined_train.csv
 combined_val.csv
 extract_embedding_vectors_albert.py
 extract_embedding_vectors_probfd.py
 extracting_vectors_albert.ipynb
 extracting_vectors_probfd.ipynb
 __pycache__
 sum_of_amino_acid_benchmark_vector.pickle
 sum_of_amino_acid_vector.pickle
'sum_of_amino_acid_vector_pickle creation-Copy1.ipynb'
'sum_of_amino_acid_vector_pickle creation.ipynb'
 Untitled.ipynb


In [9]:
!pwd


/mimer/NOBACKUP/groups/snic2019-35-21/PPI/pickle_files
